In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime as DT
import pymysql
import threading
import time

In [ ]:
mainUrl='https://www.nerjstal.ru/catalog/nerzhaveyka/truba/'
url2='https://www.nerjstal.ru'

In [ ]:
class Tube():
    name=None
    marka=None
    diam=None
    thikness=None

In [ ]:
ids=[]
resp=requests.get(mainUrl)
pageInfo=BeautifulSoup(resp.content,'lxml')
for pi in pageInfo.findAll('div',class_='catalog-list-item__name tb__cell'):
    ids.append(pi.find('a').get('href'))

for i in range(2,64):
    r=requests.get(mainUrl+'page'+str(i)+'/')
    pi=BeautifulSoup(resp.content,'lxml')
    for pi in pageInfo.findAll('div',class_='catalog-list-item__name tb__cell'):
        ids.append(pi.find('a').get('href'))

In [ ]:
def mainParser2(id_):
    r=requests.get(url2+id_)
    tubeInfo=BeautifulSoup(r.content,'lxml')
    t=Tube()
    t.name=tubeInfo.find('div',class_='page-title').text
    t.marka=tubeInfo.find('div',class_='tabs__item tabs__item_visible tabs-item').findAll('div',class_='tb__row')[2].findAll('div',class_='tb__cell')[1].text
    t.diam=tubeInfo.find('div',class_='tabs__item tabs__item_visible tabs-item').findAll('div',class_='tb__row')[3].findAll('div',class_='tb__cell')[1].text
    t.thikness=tubeInfo.find('div',class_='tabs__item tabs__item_visible tabs-item').findAll('div',class_='tb__row')[4].findAll('div',class_='tb__cell')[1].text
    tubes.append(t)
    return t

Один поток

In [ ]:
tubes_1=[]
start = time.time()
for i in ids:
    tubes_1.append(mainParser2(i))
print(f'Threading: {time.time() - start : .2f} seconds')

Многопоточность

In [ ]:
tubes_2=[]

def addTiList(a):
    tubes_2.append(mainParser2(a))

start = time.time()
threads = []
for id_ in ids:
    thread = threading.Thread(target=addTiList, args=(id_,))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
print(f'Threading: {time.time() - start : .2f} seconds')

In [ ]:
len(tubes_2)

In [ ]:
for i in tubes_2:
    print(i.name, i.marka,i.diam,i.thikness)

In [ ]:
con = pymysql.connect('localhost','root','root','profiles')
cursor = con.cursor()

In [ ]:
createTable='CREATE TABLE tubes ('+\
    'id INT NOT NULL,'+\
    'name VARCHAR(100),'+\
    'marka VARCHAR(100),'+\
    'diametr VARCHAR(100),'+\
    'thikness VARCHAR(100),'+\
    'PRIMARY KEY(id))'

In [ ]:
cursor.execute(createTable)

In [ ]:
insertSQL='INSERT INTO companies (id, name, marka,  diametr, thikness) VALUES (%s, %s, %s, %s, %s)'

In [ ]:
for i in tubes_2:
    cursor.execut(insertSQL,(i.id_,i.name,i.marka,i.diametr,i.thikness))
    con.commit()
